In [1]:
conda list


# packages in environment at /home/shyam/anaconda3/envs/DL_test:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
_openmp_mutex             4.5                       1_gnu  
argon2-cffi               20.1.0           py39h27cfd23_1  
async_generator           1.10               pyhd3eb1b0_0  
attrs                     21.4.0             pyhd3eb1b0_0  
backcall                  0.2.0              pyhd3eb1b0_0  
blas                      1.0                         mkl  
bleach                    4.1.0              pyhd3eb1b0_0  
bzip2                     1.0.8                h7b6447c_0  
ca-certificates           2021.10.26           h06a4308_2  
certifi                   2021.10.8        py39h06a4308_2  
cffi                      1.15.0           py39hd667e15_1  
cudatoolkit               10.2.89              hfd86e86_1  
dbus                      1.13.18              hb2f20db_0  
debugpy                   


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
print(os.environ['CONDA_DEFAULT_ENV'])

DL_test


In [ ]:
import torch
import numpy as np

In [ ]:
x = torch.tensor([1.,2,3])
w = torch.tensor([4.,5,6],requires_grad = True)
b = torch.tensor(5.,requires_grad = True)
torch.dot(w,x) + b

In [ ]:
y = torch.dot(w,x) + b
y


In [ ]:
y.backward()

In [92]:
print(w.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])


### Broadcasting_test


In [100]:
x = torch.randn(5,2)
y = torch.rand(2)
print(x)
print(y)
x + y

tensor([[ 1.3006,  0.9616],
        [-0.5506,  0.1331],
        [-0.7478, -0.6769],
        [-0.1376,  0.1198],
        [-0.4800,  1.9833]])
tensor([0.4057, 0.3243])


tensor([[ 1.7063,  1.2860],
        [-0.1449,  0.4575],
        [-0.3421, -0.3525],
        [ 0.2681,  0.4441],
        [-0.0743,  2.3077]])

# Linear Regression


In [57]:

inputs = np.array([[73,67,43],
                  [91,88,64],
                  [87,134,58],
                  [102,43,37],
                  [69,96,70]],dtype = 'float32')

# 5 (samples) * 3 (features)

In [58]:
targets = np.array([[56,70],[81,101],[119,133],[22,37],[103,119]],dtype  = 'float32')

# 5 (samples) * 2 (outputs)

In [59]:
### convert to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [84]:
w = torch.randn(2,3,requires_grad = True)
b = torch.randn(2,requires_grad = True)

print(w)
print(b)

tensor([[-0.2827,  1.2122, -0.0752],
        [ 2.1194,  2.3255,  0.4022]], requires_grad=True)
tensor([1.3125, 1.2541], requires_grad=True)


In [85]:
def model(x):
    return x @ w.t() + b

def mse(a,b):
    diff = a - b
    return torch.sum(diff*diff)/ diff.numel()

In [86]:
preds = model(inputs)
loss = mse(targets,preds)
loss

tensor(48793.5234, grad_fn=<DivBackward0>)

In [103]:
for i in range(100):
    preds = model(inputs)
    loss = mse(targets,preds)
    
    if loss > 0:
        loss.backward()
        with torch.no_grad():
            w -= w.grad * 1e-4
            b -= b.grad * 1e-4
            w.grad.zero_()
            b.grad.zero_()

In [104]:
preds = model(inputs)
loss = mse(targets,preds)
loss

tensor(8.0679, grad_fn=<DivBackward0>)

In [105]:
preds

tensor([[ 57.5424,  70.8018],
        [ 80.7497,  98.1439],
        [121.4048, 137.9156],
        [ 21.8233,  38.3888],
        [ 99.2118, 114.1854]], grad_fn=<AddBackward0>)

 # Linear Regression using pytorch built ins
 

In [114]:
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F


In [115]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [116]:
train_ds = TensorDataset(inputs, targets)
train_dl = DataLoader(train_ds, 2, shuffle=True)

In [117]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.3273,  0.1210,  0.2587],
        [-0.1417,  0.0386, -0.2486]], requires_grad=True)
Parameter containing:
tensor([-0.2868,  0.4878], requires_grad=True)


In [121]:
loss_fn = F.mse_loss
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [122]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            preds = model(xb)
            loss = loss_fn(preds,yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [123]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 227.5515
Epoch [20/100], Loss: 9.0792
Epoch [30/100], Loss: 3.6751
Epoch [40/100], Loss: 11.7161
Epoch [50/100], Loss: 9.5528
Epoch [60/100], Loss: 1.9342
Epoch [70/100], Loss: 6.1699
Epoch [80/100], Loss: 17.9987
Epoch [90/100], Loss: 15.8567
Epoch [100/100], Loss: 16.6525
